# TikTok Transcription Server (Colab)

This notebook acts as a remote transcription server. Run all cells to start the API.

In [1]:
# @title 1. Install Dependencies
!pip install -q yt-dlp git+https://github.com/openai/whisper.git fastapi uvicorn pyngrok nest_asyncio python-multipart
!apt-get install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
# @title 2. Load Whisper Model
import whisper
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading model on {DEVICE}...")
model = whisper.load_model("medium.en").to(DEVICE)
print("Model loaded!")

Loading model on cuda...
Model loaded!


In [5]:
# @title 3. Define Processing Logic (TikWM API Solution)
import requests
import os
import uuid
from pathlib import Path


def download_and_transcribe(video_url: str):
    """Download TikTok video using TikWM API and transcribe it"""
    unique_id = str(uuid.uuid4())
    audio_file = f"temp_{unique_id}.mp3"
    video_file = f"temp_{unique_id}.mp4"

    try:
        # 1. Download using TikWM API
        print(f"Downloading {video_url}...")

        # TikWM API endpoint
        api_url = "https://www.tikwm.com/api/"

        response = requests.post(
            api_url,
            data={
                "url": video_url,
                "hd": 1  # Request HD quality
            },
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
            }
        )

        response.raise_for_status()
        data = response.json()

        if data.get("code") != 0:
            raise ValueError(f"TikWM API error: {data.get('msg', 'Unknown error')}")

        # Get video download URL (without watermark)
        video_download_url = data["data"].get("hdplay") or data["data"].get("play")

        if not video_download_url:
            raise ValueError("No video download URL found in API response")

        # Download the video file
        print(f"Fetching video from: {video_download_url[:50]}...")
        video_response = requests.get(video_download_url, stream=True)
        video_response.raise_for_status()

        with open(video_file, 'wb') as f:
            for chunk in video_response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"✓ Downloaded: {video_file}")

        # 2. Extract audio using FFmpeg
        print(f"Extracting audio...")
        import subprocess

        ffmpeg_cmd = [
            "ffmpeg",
            "-i", video_file,
            "-vn",  # No video
            "-acodec", "libmp3lame",
            "-q:a", "2",  # High quality
            "-y",  # Overwrite
            audio_file
        ]

        result = subprocess.run(
            ffmpeg_cmd,
            capture_output=True,
            text=True
        )

        if result.returncode != 0:
            raise RuntimeError(f"FFmpeg failed: {result.stderr}")

        if not os.path.exists(audio_file):
            raise FileNotFoundError(f"Audio extraction failed: {audio_file}")

        # 3. Verify audio stream
        check_audio = subprocess.run(
            ["ffprobe", "-i", audio_file, "-show_streams", "-select_streams", "a", "-loglevel", "error"],
            capture_output=True,
            text=True
        )

        if not check_audio.stdout.strip():
            raise ValueError(f"Extracted audio has no audio stream. TikTok may not contain audio: {video_url}")

        print(f"✓ Audio extracted with verification: {audio_file}")

        # 4. Transcribe
        print(f"Transcribing {audio_file}...")
        result = model.transcribe(audio_file)
        text = result["text"]

        print(f"✓ Successfully transcribed: {video_url}")

        return text.strip()

    except Exception as e:
        print(f"Error processing {video_url}: {e}")
        raise e

    finally:
        # 5. Cleanup - always runs
        for temp_file in [audio_file, video_file]:
            try:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
                    print(f"Cleaned up: {temp_file}")
            except Exception as cleanup_error:
                print(f"Warning: Could not remove {temp_file}: {cleanup_error}")


In [ ]:
# @title 4. Start Server
import nest_asyncio
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok

# @markdown Enter your Ngrok Authtoken here:
NGROK_AUTHTOKEN = "38deD6VnibGxmEFkqkQuYJfrKSc_6Ns4ZxAwQA5WvDrbnJHkD" # @param {type:"string"}
if not NGROK_AUTHTOKEN:
    raise ValueError("Please provide an Ngrok Authtoken. Get it from https://dashboard.ngrok.com/get-started/your-authtoken")

ngrok.set_auth_token(NGROK_AUTHTOKEN)

app = FastAPI()

class VideoRequest(BaseModel):
    url: str

@app.post("/transcribe")
async def transcribe_endpoint(req: VideoRequest):
    try:
        text = download_and_transcribe(req.url)
        return {"status": "success", "text": text}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health_check()")
async def health_check():
    return {"status": "alive"}

# Open Tunnel
try:
    ngrok_tunnel = ngrok.connect(8000)
    print("\n=======================================================")
    print(f"PUBLIC URL: {ngrok_tunnel.public_url}")
    print("Copy this URL and paste it into your local Receiver App")
    print("=======================================================\n")
except Exception as e:
    print(f"Ngrok Error: {e}")
    raise e

# Run Uvicorn in the existing event loop
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
await server.serve()


PUBLIC URL: https://tartily-mensural-elaine.ngrok-free.dev
Copy this URL and paste it into your local Receiver App



INFO:     Started server process [3442]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Fetching video from: https://v16m-default.tiktokcdn.com/71c17dff177bfe0...
✓ Downloaded: temp_43b794c3-1b6a-4583-9dc9-49e1cec0d14f.mp4
Extracting audio...
✓ Audio extracted with verification: temp_43b794c3-1b6a-4583-9dc9-49e1cec0d14f.mp3
Transcribing temp_43b794c3-1b6a-4583-9dc9-49e1cec0d14f.mp3...
✓ Successfully transcribed: https://vt.tiktok.com/ZSakWTGnN/
Cleaned up: temp_43b794c3-1b6a-4583-9dc9-49e1cec0d14f.mp3
Cleaned up: temp_43b794c3-1b6a-4583-9dc9-49e1cec0d14f.mp4
INFO:     38.210.161.212:0 - "POST /transcribe HTTP/1.1" 200 OK
